In [19]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import aot

In [ ]:
# Load configuration files
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)

settings_root_path = base_dir / core_settings["paths"]["settings_path"]

In [ ]:
# AOT Test simplified parameters
total_video_number = core_settings['various']['total_video_number']  # Use total video number from original config
video_trials_per_run = 72  # Number of video trials per run
blank_trials_per_run = 12  # Number of blank trials per run
total_trials_per_run = video_trials_per_run + blank_trials_per_run

print(f"Total number of videos: {total_video_number}")
print(f"Video trials per run: {video_trials_per_run}")
print(f"Blank trials per run: {blank_trials_per_run}")
print(f"Total trials per run: {total_trials_per_run}")

总视频数: 2179
每个run的视频试次: 72
每个run的blank试次: 12
每个run的总试次数: 84


In [ ]:
# Generate list of all video names
all_video_names = []
for i in range(1, total_video_number + 1):
    video_name = str(i).zfill(4) + "_fw.mp4"  # Only use forward videos
    all_video_names.append(video_name)

print(f"Total forward videos: {len(all_video_names)}")
print(f"First few video names: {all_video_names[:10]}")

前向视频总数: 2179
前几个视频名称: ['0001_fw.mp4', '0002_fw.mp4', '0003_fw.mp4', '0004_fw.mp4', '0005_fw.mp4', '0006_fw.mp4', '0007_fw.mp4', '0008_fw.mp4', '0009_fw.mp4', '0010_fw.mp4']


In [ ]:
def generate_aottest_runs(subject_id="999"):
    """
    Generate two AOT test run configurations using the same 72 videos but in different orders
    
    Parameters:
    - subject_id: Subject ID (default 999 for testing)
    """
    
    # Randomly select 72 different forward videos (both runs use the same videos)
    selected_videos = random.sample(all_video_names, video_trials_per_run)
    
    def create_trial_list_with_blanks(videos):
        """Create trial list with blanks and shuffle order"""
        # Create trial list: 72 videos + 12 blanks
        trial_list = videos + ["blank"] * blank_trials_per_run
        
        # Shuffle trial order
        random.shuffle(trial_list)
        
        # Ensure no consecutive blanks (optional quality control)
        def avoid_consecutive_blanks(trial_list):
            trial_list = copy.deepcopy(trial_list)
            max_attempts = 100
            attempts = 0
            
            while attempts < max_attempts:
                consecutive_blanks = False
                for i in range(len(trial_list) - 1):
                    if trial_list[i] == "blank" and trial_list[i + 1] == "blank":
                        consecutive_blanks = True
                        break
                
                if not consecutive_blanks:
                    break
                    
                random.shuffle(trial_list)
                attempts += 1
                
            if attempts == max_attempts:
                print("Warning: Could not completely avoid consecutive blank trials")
                
            return trial_list
        
        return avoid_consecutive_blanks(trial_list)
    
    # Create trial lists with different orders for two runs
    trial_list_run1 = create_trial_list_with_blanks(selected_videos.copy())
    trial_list_run2 = create_trial_list_with_blanks(selected_videos.copy())
    
    # Create settings dictionaries
    settings_run1 = copy.deepcopy(stimuli_settings_temp)
    settings_run1["stimuli"]["movie_files"] = trial_list_run1
    
    settings_run2 = copy.deepcopy(stimuli_settings_temp)
    settings_run2["stimuli"]["movie_files"] = trial_list_run2
    
    # Generate filenames
    filename1 = f"experiment_settings_sub_{subject_id}_ses_aottest_run_01.yml"
    filename2 = f"experiment_settings_sub_{subject_id}_ses_aottest_run_02.yml"

    # Create save directory (if it doesn't exist)
    save_dir = settings_root_path / "aottest"
    save_dir.mkdir(parents=True, exist_ok=True)
    
    # Save files
    filepath1 = save_dir / filename1
    with open(filepath1, 'w') as outfile:
        yaml.dump(settings_run1, outfile, default_flow_style=False)

    filepath2 = save_dir / filename2
    with open(filepath2, 'w') as outfile:
        yaml.dump(settings_run2, outfile, default_flow_style=False)

    print(f"Generated config file: {filepath1}")
    print(f"Generated config file: {filepath2}")
    print(f"Both runs use the same {len(selected_videos)} videos, but in different orders")
    print(f"Total trials per run: {len(trial_list_run1)}")
    print(f"Video trials per run: {len([t for t in trial_list_run1 if t != 'blank'])}")
    print(f"Blank trials per run: {len([t for t in trial_list_run1 if t == 'blank'])}")
    
    return (settings_run1, settings_run2), (trial_list_run1, trial_list_run2), selected_videos

In [ ]:
# Generate AOT test configuration - two runs using same videos but different orders
(settings_run1, settings_run2), (trial_list_run1, trial_list_run2), selected_videos = generate_aottest_runs(subject_id="999")

已生成配置文件: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_999_ses_aottest_run_01.yml
已生成配置文件: /Users/shufanzhang/Documents/PhD/Arrow_of_time/AOTrepos/arrow_of_time_experiment/aot/data/experiment/settings/aottest/experiment_settings_sub_999_ses_aottest_run_02.yml
两个run使用相同的72个视频，但顺序不同
每个run总试次数: 84
每个run视频试次数: 72
每个run Blank试次数: 12


In [ ]:
# Display selected video list
print("Selected 72 videos:")
for i, video in enumerate(selected_videos[:10]):
    print(f"{i+1:2d}: {video}")
print("...")
for i, video in enumerate(selected_videos[-5:], len(selected_videos)-4):
    print(f"{i:2d}: {video}")

print("\n" + "="*50)
print("Run 1 - First 20 trials:")
for i, trial in enumerate(trial_list_run1[:20]):
    print(f"{i+1:2d}: {trial}")
    
print(f"\n...\n")
print("Run 1 - Last 10 trials:")
for i, trial in enumerate(trial_list_run1[-10:], len(trial_list_run1)-9):
    print(f"{i:2d}: {trial}")

print("\n" + "="*50)
print("Run 2 - First 20 trials:")
for i, trial in enumerate(trial_list_run2[:20]):
    print(f"{i+1:2d}: {trial}")
    
print(f"\n...\n")
print("Run 2 - Last 10 trials:")
for i, trial in enumerate(trial_list_run2[-10:], len(trial_list_run2)-9):
    print(f"{i:2d}: {trial}")

选中的72个视频:
 1: 0552_fw.mp4
 2: 0039_fw.mp4
 3: 1078_fw.mp4
 4: 1959_fw.mp4
 5: 0287_fw.mp4
 6: 0338_fw.mp4
 7: 0747_fw.mp4
 8: 0559_fw.mp4
 9: 0636_fw.mp4
10: 0821_fw.mp4
...
68: 0320_fw.mp4
69: 2043_fw.mp4
70: 0151_fw.mp4
71: 0970_fw.mp4
72: 0625_fw.mp4

Run 1 - 前20个试次:
 1: 1293_fw.mp4
 2: 0953_fw.mp4
 3: 1509_fw.mp4
 4: 0121_fw.mp4
 5: 0756_fw.mp4
 6: 0712_fw.mp4
 7: 1926_fw.mp4
 8: 1699_fw.mp4
 9: 0018_fw.mp4
10: 0995_fw.mp4
11: blank
12: 0697_fw.mp4
13: 0688_fw.mp4
14: 2014_fw.mp4
15: 0779_fw.mp4
16: 0700_fw.mp4
17: 0962_fw.mp4
18: 1497_fw.mp4
19: 0287_fw.mp4
20: 1959_fw.mp4

...

Run 1 - 后10个试次:
75: 0452_fw.mp4
76: 0644_fw.mp4
77: 0552_fw.mp4
78: 1570_fw.mp4
79: 0853_fw.mp4
80: blank
81: 1064_fw.mp4
82: 1710_fw.mp4
83: blank
84: 0965_fw.mp4

Run 2 - 前20个试次:
 1: 0504_fw.mp4
 2: blank
 3: 0456_fw.mp4
 4: 0747_fw.mp4
 5: 0688_fw.mp4
 6: 1505_fw.mp4
 7: blank
 8: 0552_fw.mp4
 9: 0039_fw.mp4
10: 0387_fw.mp4
11: 0821_fw.mp4
12: 0338_fw.mp4
13: 1429_fw.mp4
14: 0018_fw.mp4
15: 1401_fw.mp4


In [ ]:
# Verify results - check both runs
def verify_run(trial_list, run_name):
    video_count = len([t for t in trial_list if t != 'blank'])
    blank_count = len([t for t in trial_list if t == 'blank'])
    unique_videos = set([t for t in trial_list if t != 'blank'])
    
    print(f"{run_name}verification results:")
    print(f"✓ Total trials: {len(trial_list)} (expected: {total_trials_per_run})")
    print(f"✓ Video trials: {video_count} (expected: {video_trials_per_run})")
    print(f"✓ Blank trials: {blank_count} (expected: {blank_trials_per_run})")
    print(f"✓ Unique videos: {len(unique_videos)} (expected: {video_trials_per_run})")
    print(f"✓ All videos are forward (_fw): {all(v.endswith('_fw.mp4') for v in unique_videos)}")
    return unique_videos

print("Verification of two runs using same videos but different orders:")
print("="*60)

run1_videos = verify_run(trial_list_run1, "Run 1 ")
print()
run2_videos = verify_run(trial_list_run2, "Run 2 ")

print()
print("Video consistency check between runs:")
videos_match = run1_videos == run2_videos
print(f"✓ Both runs use exactly the same videos: {videos_match}")

if videos_match:
    print(f"✓ Number of shared videos: {len(run1_videos)}")
else:
    print(f"✗ Videos don't match!")
    print(f"  Run1 only: {run1_videos - run2_videos}")
    print(f"  Run2 only: {run2_videos - run1_videos}")

两个run使用相同视频但顺序不同的验证:
Run 1 验证结果:
✓ 总试次数: 84 (期望: 84)
✓ 视频试次: 72 (期望: 72)
✓ Blank试次: 12 (期望: 12)
✓ 独特视频数: 72 (期望: 72)
✓ 所有视频都是前向(_fw): True

Run 2 验证结果:
✓ 总试次数: 84 (期望: 84)
✓ 视频试次: 72 (期望: 72)
✓ Blank试次: 12 (期望: 12)
✓ 独特视频数: 72 (期望: 72)
✓ 所有视频都是前向(_fw): True

两个run视频一致性检查:
✓ 两个run使用完全相同的视频: True
✓ 共同使用的视频数量: 72


In [ ]:
# Check for consecutive blanks in both runs
def check_consecutive_blanks(trial_list, run_name):
    consecutive_blanks = []
    for i in range(len(trial_list) - 1):
        if trial_list[i] == "blank" and trial_list[i + 1] == "blank":
            consecutive_blanks.append((i, i+1))

    if consecutive_blanks:
        print(f"Warning: {run_name}found {len(consecutive_blanks)} consecutive blanks:")
        for start, end in consecutive_blanks:
            print(f"  Position {start+1}-{end+1}")
    else:
        print(f"✓ {run_name}no consecutive blank trials")
    
    return len(consecutive_blanks)

print("Consecutive blank check:")
print("="*30)
run1_consecutive = check_consecutive_blanks(trial_list_run1, "Run 1 ")
run2_consecutive = check_consecutive_blanks(trial_list_run2, "Run 2 ")

print(f"\nSummary: Both runs avoid consecutive blanks: {run1_consecutive == 0 and run2_consecutive == 0}")

连续blank检查:
✓ Run 1 没有连续的blank试次
✓ Run 2 没有连续的blank试次

总结: 两个run都避免了连续blank: True
